In [1]:
# Завантаження даних (може відрізнятись в залежності від оточення в якому ви працюєте)

import pandas as pd

# Шлях до файлу (вкажіть ваш власний шлях до файлу)
file_path = '/kaggle/input/data-prep/train_data.csv'

# Завантаження даних
data = pd.read_csv(file_path)

# Перевірка перших кількох рядків
print(data.head())

                 id                               cleaned_comment_text  \
0  0000997932d777bf  Explanation Why the edits made under my userna...   
1  000103f0d9cfb60f  Daww He matches this background colour Im seem...   
2  000113f07ec002fd  Hey man Im really not trying to edit war Its j...   
3  0001b41b1c6bb37e  More I cant make any real suggestions on impro...   
4  0001d958c54c6e35  You sir are my hero Any chance you remember wh...   

                                           input_ids  \
0  [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...   
1  [101, 4830, 2860, 2860, 2002, 3503, 2023, 4281...   
2  [101, 4931, 2158, 10047, 2428, 2025, 2667, 200...   
3  [101, 2062, 1045, 2064, 2102, 2191, 2151, 2613...   
4  [101, 2017, 2909, 2024, 2026, 5394, 2151, 3382...   

                                     attention_masks  toxic  severe_toxic  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      0             0   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      0     

In [2]:
# виводимо загальну інформацію про вибірку
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    159571 non-null  object
 1   cleaned_comment_text  159571 non-null  object
 2   input_ids             159571 non-null  object
 3   attention_masks       159571 non-null  object
 4   toxic                 159571 non-null  int64 
 5   severe_toxic          159571 non-null  int64 
 6   obscene               159571 non-null  int64 
 7   threat                159571 non-null  int64 
 8   insult                159571 non-null  int64 
 9   identity_hate         159571 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 12.2+ MB


In [3]:
# перевіряємо унікальні значення для кожного класу

# Підрахунок кількості позитивних прикладів для кожного класу
class_distribution = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print("Кількість позитивних прикладів для кожного класу:\n", class_distribution)

# Загальна кількість прикладів
total_samples = len(data)

# Відсоткове співвідношення по кожному класу
class_percentage = (class_distribution / total_samples) * 100
print("\nВідсоткове співвідношення класів:\n", class_percentage)

# Підрахунок нетоксичних прикладів
non_toxic_count = (data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) == 0).sum()
non_toxic_percentage = (non_toxic_count / total_samples) * 100

# Виведення нетоксичних прикладів
print(f"\nКількість нетоксичних прикладів: {non_toxic_count}")
print(f"Відсоток нетоксичних прикладів: {non_toxic_percentage:.2f}%")

Кількість позитивних прикладів для кожного класу:
 toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

Відсоткове співвідношення класів:
 toxic            9.584448
severe_toxic     0.999555
obscene          5.294822
threat           0.299553
insult           4.936361
identity_hate    0.880486
dtype: float64

Кількість нетоксичних прикладів: 143346
Відсоток нетоксичних прикладів: 89.83%


**Створення та підготовка міні-датасету для тестового моделювання**

Якщо ви вирішили не створювати міні вибірку і працювати одразу із повним файлом, пропустіть цей блок

In [4]:
# Створюємо копію даних для міні-вибірки
data_copy = data.copy()

# Створення колонки label_combination
data_copy['label_combination'] = data_copy[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].apply(
    lambda row: '-'.join(row.astype(str)), axis=1
)

# Видалення рідкісних комбінацій міток у копії
label_counts = data_copy['label_combination'].value_counts()
data_copy = data_copy[data_copy['label_combination'].isin(label_counts[label_counts > 1].index)].copy()

# Видалення тимчасової колонки
data_copy.drop(columns=['label_combination'], inplace=True)

# Виділення міні-вибірки (10% даних)
from sklearn.model_selection import train_test_split

data_sample, _ = train_test_split(
    data_copy,
    test_size=0.9,  # 10% в міні-вибірку
    stratify=data_copy[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    random_state=42
)

print(f"Розмір міні-вибірки: {len(data_sample)}")

Розмір міні-вибірки: 15956


In [5]:
# Розмір вибірки все же завеликий для тестових цілей, сменшуємо ще
# Зменшуємо розмір вибірки до 10% без втрати рідкісних класів
smaller_sample_size = int(len(data_sample) * 0.1)

# Вибірка випадкових рядків без порушення пропорцій
data_sample = data_sample.sample(n=smaller_sample_size, random_state=42)

print(f"Кількість прикладів у новій міні-вибірці: {len(data_sample)}")

# Перевіряємо, чи збережені рідкісні класи
class_distribution = data_sample[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print("Розподіл класів у зменшеній вибірці:\n", class_distribution)


Кількість прикладів у новій міні-вибірці: 1595
Розподіл класів у зменшеній вибірці:
 toxic            153
severe_toxic      21
obscene           96
threat             8
insult            80
identity_hate     16
dtype: int64


In [6]:
import tensorflow as tf

# Перетворення input_ids та attention_masks у списки для міні-вибірки
mini_input_ids = tf.convert_to_tensor(data_sample['input_ids'].apply(eval).tolist())
mini_attention_masks = tf.convert_to_tensor(data_sample['attention_masks'].apply(eval).tolist())

# Перетворення labels у тензор для міні-вибірки
mini_labels = tf.convert_to_tensor(data_sample[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values)

# Перевірка формату міні-вибірки
print(f"mini_input_ids shape: {mini_input_ids.shape}")
print(f"mini_attention_masks shape: {mini_attention_masks.shape}")
print(f"mini_labels shape: {mini_labels.shape}")

mini_input_ids shape: (1595, 128)
mini_attention_masks shape: (1595, 128)
mini_labels shape: (1595, 6)


In [7]:
# Створюємо tf.data.Dataset для міні-вибірки
mini_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': mini_input_ids,
    'attention_mask': mini_attention_masks
}, mini_labels))

# Перевірка кількості прикладів у міні-вибірці
print(f"Кількість прикладів у mini_dataset: {len(mini_dataset)}")

Кількість прикладів у mini_dataset: 1595


In [8]:
# Перевірка структури mini_dataset
for X, Y in mini_dataset.take(1):  # Перевіряємо структуру
    print(f"X (input_ids): {X['input_ids'].shape}")
    print(f"X (attention_mask): {X['attention_mask'].shape}")
    print(f"Y (labels): {Y.shape}")

X (input_ids): (128,)
X (attention_mask): (128,)
Y (labels): (6,)


In [9]:
'''
розбиття вибірки на тренувальну і валідаційну.
Оскільки на цьому етапі ми виконуємо тестову побудову моделі і її точність нам не важлива, 
розділимо вибірку "грубо", ігноруючи диспропорції класів. При роботі із повною вибіркою у
наступних частинах коду цей підхід не припустимий та має бути змінений.
'''
# Розподіл mini_dataset на тренувальну та валідаційну вибірки
validation_split = 0.2  # 20% для валідації
total_size = len(mini_dataset)
val_size = int(total_size * validation_split)

# Використовуємо take() і skip() для поділу
mini_val_dataset = mini_dataset.take(val_size)
mini_train_dataset = mini_dataset.skip(val_size)

# Перевірка розмірів
print(f"Тренувальна вибірка: {len(mini_train_dataset)} прикладів")
print(f"Валідаційна вибірка: {len(mini_val_dataset)} прикладів")

Тренувальна вибірка: 1276 прикладів
Валідаційна вибірка: 319 прикладів


In [10]:
# Мінімальний тест, щоб переконатись, що модель приймає оброблені дані
from transformers import TFBertModel

# Завантажуємо предобучену модель BERT
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Передаємо батч з розміром 1 у модель
for batch in mini_train_dataset.take(1):  # Беремо перший батч
    mini_inputs, mini_labels = batch
    mini_input_ids = tf.expand_dims(mini_inputs['input_ids'], axis=0)  # Додаємо розмір батчу
    mini_attention_mask = tf.expand_dims(mini_inputs['attention_mask'], axis=0)  # Додаємо розмір батчу

    # Передаємо дані у модель
    outputs = bert_model(input_ids=mini_input_ids, attention_mask=mini_attention_mask)
    print("Модель успішно прийняла дані!")
    print(f"Вихідний shape: {outputs.last_hidden_state.shape}")
    break

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Модель успішно прийняла дані!
Вихідний shape: (1, 128, 768)


**Створення та підготовка повного датасету**

In [15]:
from sklearn.model_selection import train_test_split

# Створюємо копію даних для розділення
data_copy = data.copy()

# Створення колонки label_combination
data_copy['label_combination'] = data_copy[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].apply(
    lambda row: '-'.join(row.astype(str)), axis=1
)

# Видалення рідкісних комбінацій міток у копії
label_counts = data_copy['label_combination'].value_counts()
data_copy = data_copy[data_copy['label_combination'].isin(label_counts[label_counts > 1].index)].copy()

# Видалення тимчасової колонки
data_copy.drop(columns=['label_combination'], inplace=True)

# Розділення на тренувальну і валідаційну вибірки
train_data, val_data = train_test_split(
    data_copy,
    test_size=0.2,  # 20% для валідації
    stratify=data_copy[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    random_state=42
)

# Перевірка розмірів
print(f"Тренувальна вибірка: {len(train_data)} прикладів")
print(f"Валідаційна вибірка: {len(val_data)} прикладів")


Тренувальна вибірка: 127655 прикладів
Валідаційна вибірка: 31914 прикладів


In [16]:
# перевіряємо унікальні значення для кожного класу для тренувальної вибірки

# Підрахунок кількості позитивних прикладів для кожного класу
class_distribution = train_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print("Кількість позитивних прикладів для кожного класу:\n", class_distribution)

# Загальна кількість прикладів
total_samples = len(train_data)

# Відсоткове співвідношення по кожному класу
class_percentage = (class_distribution / total_samples) * 100
print("\nВідсоткове співвідношення класів:\n", class_percentage)

# Підрахунок нетоксичних прикладів
non_toxic_count = (train_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) == 0).sum()
non_toxic_percentage = (non_toxic_count / total_samples) * 100

# Виведення нетоксичних прикладів
print(f"\nКількість нетоксичних прикладів: {non_toxic_count}")
print(f"Відсоток нетоксичних прикладів: {non_toxic_percentage:.2f}%")

Кількість позитивних прикладів для кожного класу:
 toxic            12233
severe_toxic      1274
obscene           6759
threat             382
insult            6300
identity_hate     1122
dtype: int64

Відсоткове співвідношення класів:
 toxic            9.582860
severe_toxic     0.998002
obscene          5.294740
threat           0.299244
insult           4.935177
identity_hate    0.878931
dtype: float64

Кількість нетоксичних прикладів: 114677
Відсоток нетоксичних прикладів: 89.83%


In [17]:
# перевіряємо унікальні значення для кожного класу для валідаційної вибірки

# Підрахунок кількості позитивних прикладів для кожного класу
class_distribution = val_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print("Кількість позитивних прикладів для кожного класу:\n", class_distribution)

# Загальна кількість прикладів
total_samples = len(val_data)

# Відсоткове співвідношення по кожному класу
class_percentage = (class_distribution / total_samples) * 100
print("\nВідсоткове співвідношення класів:\n", class_percentage)

# Підрахунок нетоксичних прикладів
non_toxic_count = (val_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) == 0).sum()
non_toxic_percentage = (non_toxic_count / total_samples) * 100

# Виведення нетоксичних прикладів
print(f"\nКількість нетоксичних прикладів: {non_toxic_count}")
print(f"Відсоток нетоксичних прикладів: {non_toxic_percentage:.2f}%")

Кількість позитивних прикладів для кожного класу:
 toxic            3059
severe_toxic      319
obscene          1690
threat             94
insult           1576
identity_hate     282
dtype: int64

Відсоткове співвідношення класів:
 toxic            9.585135
severe_toxic     0.999561
obscene          5.295482
threat           0.294542
insult           4.938272
identity_hate    0.883625
dtype: float64

Кількість нетоксичних прикладів: 28669
Відсоток нетоксичних прикладів: 89.83%


Вибірка розподілена рівномірно з врахуванням пропорцій класів.

In [19]:
# Перетворення тренувальної вибірки

import tensorflow as tf

# Перетворення input_ids та attention_masks у списки
input_ids = tf.convert_to_tensor(train_data['input_ids'].apply(eval).tolist())
attention_masks = tf.convert_to_tensor(train_data['attention_masks'].apply(eval).tolist())

# Перетворення labels у тензор
labels = tf.convert_to_tensor(train_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values)

# Перевірка формату
print(f"input_ids shape: {input_ids.shape}")
print(f"attention_masks shape: {attention_masks.shape}")
print(f"labels shape: {labels.shape}")

# Створюємо tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': input_ids,
    'attention_mask': attention_masks
}, labels))

# Перевірка кількості прикладів
print(f"Кількість прикладів у dataset: {len(train_dataset)}")

# Перевірка структури train_dataset
for X, Y in train_dataset.take(1):  # Перевіряємо структуру
    print(f"X (input_ids): {X['input_ids'].shape}")
    print(f"X (attention_mask): {X['attention_mask'].shape}")
    print(f"Y (labels): {Y.shape}")

input_ids shape: (127655, 128)
attention_masks shape: (127655, 128)
labels shape: (127655, 6)
Кількість прикладів у dataset: 127655
X (input_ids): (128,)
X (attention_mask): (128,)
Y (labels): (6,)


In [20]:
# Перетворення валідаціної вибірки

# Перетворення input_ids та attention_masks у списки для валідаційної вибірки
val_input_ids = tf.convert_to_tensor(val_data['input_ids'].apply(eval).tolist())
val_attention_masks = tf.convert_to_tensor(val_data['attention_masks'].apply(eval).tolist())

# Перетворення labels у тензор для валідаційної вибірки
val_labels = tf.convert_to_tensor(val_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values)

# Перевірка формату
print(f"val_input_ids shape: {val_input_ids.shape}")
print(f"val_attention_masks shape: {val_attention_masks.shape}")
print(f"val_labels shape: {val_labels.shape}")

# Створюємо tf.data.Dataset для валідаційної вибірки
val_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': val_input_ids,
    'attention_mask': val_attention_masks
}, val_labels))

# Перевірка кількості прикладів
print(f"Кількість прикладів у val_dataset: {len(val_dataset)}")

# Перевірка структури val_dataset
for X, Y in val_dataset.take(1):  # Перевіряємо структуру
    print(f"X (val_input_ids): {X['input_ids'].shape}")
    print(f"X (val_attention_mask): {X['attention_mask'].shape}")
    print(f"Y (val_labels): {Y.shape}")


val_input_ids shape: (31914, 128)
val_attention_masks shape: (31914, 128)
val_labels shape: (31914, 6)
Кількість прикладів у val_dataset: 31914
X (val_input_ids): (128,)
X (val_attention_mask): (128,)
Y (val_labels): (6,)


In [21]:
# Мінімальний тест, щоб переконатись, що модель приймає оброблені дані
from transformers import TFBertModel

# Завантажуємо предобучену модель BERT
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Передаємо батч з розміром 1 у модель
for batch in train_dataset.take(1):  # Беремо перший батч
    inputs, labels = batch
    input_ids = tf.expand_dims(inputs['input_ids'], axis=0)  # Додаємо розмір батчу
    attention_mask = tf.expand_dims(inputs['attention_mask'], axis=0)  # Додаємо розмір батчу

    # Передаємо дані у модель
    outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
    print("Модель успішно прийняла дані!")
    print(f"Вихідний shape: {outputs.last_hidden_state.shape}")
    break

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Модель успішно прийняла дані!
Вихідний shape: (1, 128, 768)


**Примітка**

дані підготовлені для роботи із моделлю БЕРТ. Але через конфлікти бібліотек PyTorch / Tensorflow можливо доведеться в коді зі створення та навчання моделі виконувати додаткові перетворення. 
У власному коді для вирішення кофліктів я використовувала створений кастомний клас даних. Він наданий нижче для прикладу. У вашому коді можливо ви вирішите конфлікти іншими шляхами.

In [22]:
from transformers import TFBertModel
from tensorflow.keras.layers import Input, Layer
from tensorflow.keras.models import Model
import tensorflow as tf

# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state